In [13]:
# Geographic Data Handling
import geopandas as gpd
import shapely.geometry as sg

# Network Analysis and Routing
import osmnx as ox

# Data Manipulation and Analysis
import numpy as np
from numpy import zeros, column_stack
import pandas as pd

# Time and Date Handling
from time import time
from datetime import timedelta

# File and directory operations
import os

# Mathematical Functions
from math import exp, hypot

# Raster and Image Processing
from rasterio.transform import rowcol, xy
from skimage.draw import line, disk, circle_perimeter
import rasterio
from osgeo import gdal

# Progress Tracking
from tqdm.auto import tqdm

In [14]:
# Input data
filepath = "C:/Users/ygrin/Documents/Studie - MSc ADS/Utrecht University/Block 4 - Thesis/TestData/"
multi_point_file = filepath+"Test_multiple_home_locations.gpkg"
single_point_file = filepath+"Test_single_home_location.gpkg"
results_path = "C:/Users/ygrin/Documents/Studie - MSc ADS/Utrecht University/Block 4 - Thesis/TestData/Results/"

In [3]:
import pyproj
# OSM NETWORK
epsg_transformer = pyproj.Transformer.from_crs("epsg:27700", "epsg:4326")
latlon = epsg_transformer.transform(single_point['geometry'].x,single_point['geometry'].y)
network_osm = ox.graph_from_point(latlon, dist=250)
network_osm_bng = ox.project_graph(network_osm, to_crs='EPSG:27700')
network_osm_bng_edges = ox.graph_to_gdfs(network_osm_bng, nodes=False, edges=True)
network_osm_bng_edges_index = network_osm_bng_edges.reset_index(drop=True)

c:\Users\ygrin\anaconda3\envs\ADSthesis\Lib\site-packages\pyproj\transformer.py:812: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return self._transformer._transform_point(


# GVI functions
Original code: https://github.com/jonnyhuck/green-visibility-index/blob/master/gvi.py

In [4]:
def coords2Array(a, x, y):
    """
    * convert between coords and array position
    *  returns row,col (y,x) as expected by rasterio
    """
    r, c = rowcol(a, x, y)
    return int(r), int(c)


def array2Coords(a, row, col):
    """
    * convert between array position and coords
    *  params are row,col (y,x) as expected by rasterio
    *  returns coords at the CENTRE of the cell
    """
    x, y = xy(a, row, col)
    return int(x), int(y)


def viewshed(r0, c0, radius_px, resolution, observerHeight, targetHeight, dsm_data, dtm_data, a):
    """
    * Use Bresenham's Circle / Midpoint algorithm to determine endpoints for viewshed
    """

    # create output array at the same dimensions as data for viewshed
    output = zeros(dtm_data.shape)

    # set the start location as visible automatically
    output[(r0, c0)] = 1

    # get pixels in the circle
    for r, c in column_stack(circle_perimeter(r0, c0, radius_px)):

        # calculate line of sight to each pixel
        output = lineOfSight(r0, c0, r, c, resolution, observerHeight, targetHeight, dsm_data, dtm_data, output)

    # return the resulting viewshed
    return output

def lineOfSight(r0, c0, r1, c1, observer_height, resolution, target_height, dsm_data, dtm_data, output):
    """
     * Runs a single ray-trace from one point to another point, returning a list of visible cells
    """

    # init variables for loop
    cur_dydx = 0 		  	# current dydx (base of object)
    max_dydx = 0 	  		# biggest dydx so far
    # top_dydx = 0 		    # current dydx (top of object)
    distance_travelled = 0  # how far we have travelled along the ray

    # get the viewer height
    height0 = dtm_data[(r0, c0)] + observer_height

    # get the pixels in the line (excluding the first one	)
    pixels = column_stack(line(r0, c0, r1, c1))[1:]

    # loop along the pixels in the line
    for r, c in pixels:

        # distance travelled so far
        distance_travelled = hypot(c0 - c, r0 - r)

        ''' comment this out as long as we use 0 as target offset '''
        ## set cell as visible if the height of the top of the object from the DTM > previous max
        # top_dydx = (dsm_data[(r, c)] - height0 + target_height) / distance_travelled
        # if (top_dydx >= max_dydx):
        # 	output[(r, c)] = 1
        #
        ## update max dydx the height of the base of the object on the DSM > previous max
        # cur_dydx = (dsm_data[(r, c)] - height0) / distance_travelled
        # if (cur_dydx > max_dydx):
        # 	max_dydx = cur_dydx

        # update max dydx the height of the base of the object on the DSM > previous max
        cur_dydx = (dsm_data[(r, c)] - height0) / (distance_travelled * resolution)
        if (cur_dydx > max_dydx):
            max_dydx = cur_dydx
            output[(r, c)] = 1

    # return updated output surface
    return output


def calc_gvi(mask, df_row, geom_type):
    # create an output array at the same dimensions as data for output
    gvi = zeros((mask["meta"]["height"], mask["meta"]["width"]))

    # radius in pixels
    radius_px = int(mask["options"]["radius"] // mask['meta']['transform'][0])

    # build weighting mask
    weighting_mask = zeros((radius_px*2, radius_px*2))
    for r, c in column_stack(disk((radius_px, radius_px), radius_px, shape=weighting_mask.shape)):
        weighting_mask[(r, c)] = exp(-0.0003 * (hypot(radius_px - c, radius_px - r) * mask['meta']['transform'][0]))

    # determine nr. of points upon which average GVI will be based for each geometry in poi file
    nr_of_points = len(df_row['sampled_points'])
    point_list = df_row['sampled_points']
    
    gvi_values = []
    for point in tqdm(point_list, desc=f'Calculating GVI for {geom_type} {df_row.id}'):
        r,c  = coords2Array(mask["meta"]["transform"], point.x, point.y)

        # call (weighted) viewshed
        output = viewshed(r, c, radius_px, 		# coords and radius in pixels
            mask['meta']['transform'][0],		# resolution of datasets
            mask["options"]["o_height"], 		# observer height
            mask["options"]["t_height"],		# target height
            mask["dsm"], 						# dsm dataset
            mask["dtm"],						# dtm dataset
            mask["meta"]["transform"])			# affine transform

        # extract the viewshed data from the output surface and apply weighting mask
        visible = output[r-radius_px:r+radius_px, c-radius_px:c+radius_px] * weighting_mask

        # multiply extract of (weighted) viewshed with extract of (weighted) green dataset
        visible_green = visible * (mask["green"][r-radius_px:r+radius_px, c-radius_px:c+radius_px] * weighting_mask)

        # get the ratio for greenness in the view
        gvi = visible_green.sum() / visible.sum()
        gvi_values.append(gvi)
    
    # Return the mean GVI, nr_of_points and the list of GVI values that correspond to the sampled road locations
    return np.mean(gvi_values).round(3), nr_of_points, gvi_values

# TOP LEVEL FUNCTION

In [5]:
# Get sample points within sub-network to calculate GVI
def get_network_sample_points(df_row, network_edges, buffer_dist, sample_dist):
    # Create sub-network based on poi geometry and buffer_dist in case provided
    if buffer_dist is None:
        buffer_edges = network_edges[network_edges.intersects(df_row['geometry'])].reset_index(drop=True)
    else:
        buffer_edges = network_edges[network_edges.intersects(df_row['geometry'].buffer(buffer_dist))].reset_index(drop=True)
    
    # Get length of network edges
    edges_length_meters = buffer_edges.length
    edges_length_numeric = edges_length_meters.astype(float)
    
    sampled_points = []
    # If edge length < sample_dist, compute edge centroid and save geometry, else; interpolate edge and get geometries for each sample_dist meters
    for i in range(len(buffer_edges)):
        if edges_length_numeric[i] < sample_dist:
            point = buffer_edges.geometry[i].centroid
            sampled_points.append(point)
        else:
            line = buffer_edges.geometry[i]
            num_points = int(line.length / sample_dist) + 1  # use this instead of distances to ensure more uniform sampling
            distances = np.linspace(0, line.length, num=num_points)
            points = [line.interpolate(distance) for distance in distances]
            sampled_points.extend(points)

    return sampled_points

In [6]:
def get_viewshed_GVI(point_of_interest, greendata_raster_file, dtm_raster_file, dsm_raster_file, network_file=None, crs_epsg=None, 
                     polygon_type="neighbourhood", buffer_dist=None, viewing_dist=250, sample_dist=50, observer_height=1.7, write_to_file=True, 
                     output_dir=os.getcwd()):
    ### Step 1: Read and process user inputs, check conditions
    if isinstance(point_of_interest, gpd.GeoDataFrame):
        poi = point_of_interest
    else:
        poi = gpd.read_file(point_of_interest)

    # Make sure geometries of poi file are either all provided using point geometries or all using polygon geometries
    if all(poi['geometry'].geom_type == 'Point') or all(poi['geometry'].geom_type == 'Polygon') or all(poi['geometry'].geom_type == 'MultiPolygon'):
        geom_type = poi.iloc[0]['geometry'].geom_type
    else:
        raise TypeError("Please make sure all geometries are of 'Point' type, all geometries are of 'Polygon' type or all geometries are of 'MultiPolygon' type and re-run the function")

    # Make sure CRS of poi file is projected rather than geographic
    if not poi.crs.is_projected:
        if crs_epsg is None:
            print("Warning: The CRS of the PoI dataset is currently geographic, therefore it will now be projected to CRS with EPSG:3395")
            epsg = 3395
            poi.to_crs(f"EPSG:{epsg}", inplace=True)
        else:
            print(f"Warning: The CRS of the PoI dataset is currently geographic, therefore it will now be projected to EPSG:{crs_epsg} as specified")
            epsg = crs_epsg
            poi.to_crs(f"EPSG:{epsg}", inplace=True)
    else:
        epsg = poi.crs.to_epsg()

    # In case of house polygons, transform to centroids
    if geom_type == "Polygon" or geom_type == "MultiPolygon":
        if polygon_type not in ["neighbourhood", "house"]:
            raise ValueError("Please make sure that the polygon_type argument is set to either 'neighbourhood' or 'house'")
        if polygon_type == "house":
            print("Changing geometry type to Point by computing polygon centroids...")
            poi['geometry'] = poi['geometry'].centroid
            geom_type = poi.iloc[0]['geometry'].geom_type
            print("Done \n")

    # Make sure poi file contains ID columns to identify unique locations
    if "id" in poi.columns:
        if poi['id'].isnull().values.any():
            poi['id'] = poi['id'].fillna(pd.Series(range(1, len(poi) + 1))).astype(int)
    else:
        poi['id'] = pd.Series(range(1, len(poi) + 1)).astype(int)

    # Validate user inputs
    if geom_type == "Point":
        if not isinstance(buffer_dist, int) or (not buffer_dist > 0):
            raise ValueError("Please make sure that the buffer_dist argument is set to a positive integer")
    
    # Make sure viewing_dist is set 
    if not isinstance(viewing_dist, int) or (not viewing_dist > 0):
        raise ValueError("Please make sure that the viewing_dist argument is set to a positive integer")

    # Make sure sample_dist is set 
    if not isinstance(sample_dist, (float, int)) or (not sample_dist > 0):
        raise ValueError("Please make sure that the sample_dist argument is set to a positive number")
    
    # Make sure observer_height is set
    if not isinstance(observer_height, (float, int)) or (not observer_height > 0):
        raise ValueError("Please make sure that the observer_height argument is set to a positive number")

    # Read DSM, DTM and greenspace rasters
    with rasterio.open(dsm_raster_file) as src:
        dsm = src.read(1)
        dsm_crs = src.crs.to_epsg()
        if dsm_crs is None:
            raise ValueError("The DSM raster does not have a CRS, please make sure it does and re-run the function")
        dsm_bounds = src.bounds

    # Reproject if EPSG is not equal to CRS of poi file
    if not dsm_crs == epsg:
        print("Reprojecting the DSM file so that the CRS matches the CRS of the poi file...")
        gdal.Warp('/vsimem/reprojected.tif', dsm_raster_file, srcSRS=f"EPSG:{dsm_crs}", dstSRS=f"EPSG:{epsg}")
        with rasterio.open('/vsimem/reprojected.tif') as src:
            dsm = src.read(1)
            dsm_bounds = src.bounds
        gdal.Unlink('/vsimem/reprojected.tif')
        print("Done \n")

    # Make sure all points of interest are within or do at least intersect (in case of polygons) the DSM raster provided
    if not all(geom.within(sg.box(*dsm_bounds)) for geom in poi['geometry']):
        if geom_type == "Point":
            raise ValueError("Not all points of interest are within the DSM file provided, please make sure they are and re-run the function")
        else:
            if not all(geom.intersects(sg.box(*dsm_bounds)) for geom in poi['geometry']):
                raise ValueError("Not all polygons of interest are within, or do at least partly intersect, with the area covered by the DSM file provided, please make sure they are/do and re-run the function")
            else:
                print("Warning: Not all polygons of interest are completely within the area covered by the DSM file provided, results will be based on intersecting part of polygons involved \n")
    
    with rasterio.open(dtm_raster_file) as src:
        dtm = src.read(1)
        dtm_crs = src.crs.to_epsg()
        if dtm_crs is None:
            raise ValueError("The DTM raster does not have a CRS, please make sure it does and re-run the function")
        dtm_bounds = src.bounds
    
    if not dtm_crs == epsg:
        print("Reprojecting the DTM file so that the CRS matches the CRS of the poi file...")
        gdal.Warp('/vsimem/reprojected.tif', dtm_raster_file, srcSRS=f"EPSG:{dtm_crs}", dstSRS=f"EPSG:{epsg}")
        with rasterio.open('/vsimem/reprojected.tif') as src:
            dtm = src.read(1)
            dtm_bounds = src.bounds
        gdal.Unlink('/vsimem/reprojected.tif')
        print("Done \n")
    
    # Make sure all points of interest are within or do at least intersect (in case of polygons) the DTM raster provided
    if not all(geom.within(sg.box(*dtm_bounds)) for geom in poi['geometry']):
        if geom_type == "Point":
            raise ValueError("Not all points of interest are within the DTM file provided, please make sure they are and re-run the function")
        else:
            if not all(geom.intersects(sg.box(*dtm_bounds)) for geom in poi['geometry']):
                raise ValueError("Not all polygons of interest are within, or do at least partly intersect, with the area covered by the DTM file provided, please make sure they are/do and re-run the function")
            else:
                print("Warning: Not all polygons of interest are completely within the area covered by the DTM file provided, results will be based on intersecting part of polygons involved \n")
    
    # Create metadata for object that will be passed into viewshed GVI function
    meta = {
        'height': dtm.shape[0],
        'width': dtm.shape[1],
        'transform': src.transform,
        "driver": "GTiff",
        'count': 1,
        "crs": src.crs
    }

    with rasterio.open(greendata_raster_file) as src:
        green = src.read(1)
        green_crs = src.crs.to_epsg()
        if green_crs is None:
            raise ValueError("The greenspace raster does not have a CRS, please make sure it does and re-run the function")
        green_bounds = src.bounds
    
    if not green_crs == epsg:
        print("Reprojecting the greenspace file so that the CRS matches the CRS of the poi file...")
        gdal.Warp('/vsimem/reprojected.tif', greendata_raster_file, srcSRS=f"EPSG:{green_crs}", dstSRS=f"EPSG:{epsg}")
        with rasterio.open('/vsimem/reprojected.tif') as src:
            green = src.read(1)
            green_bounds = src.bounds
        gdal.Unlink('/vsimem/reprojected.tif')
        print("Done \n")
        
    # Make sure all points of interest are within or do at least intersect (in case of polygons) the Greenspace raster provided
    if not all(geom.within(sg.box(*green_bounds)) for geom in poi['geometry']):
        if geom_type == "Point":
            raise ValueError("Not all points of interest are within the Greenspace file provided, please make sure they are and re-run the function")
        else:
            if not all(geom.intersects(sg.box(*green_bounds)) for geom in poi['geometry']):
                raise ValueError("Not all polygons of interest are within, or do at least partly intersect, with the area covered by the Greenspace file provided, please make sure they are/do and re-run the function")
            else:
                print("Warning: Not all polygons of interest are completely within the area covered by the Greenspace file provided, results will be based on intersecting part of polygons involved \n")

    # Specify variables that will be used in GVI function
    options = {
        'radius': viewing_dist, 
        'o_height': observer_height,  # 1.7 meters (typical height of a person)
        't_height': 0  # 0 meters (the target is the ground)
    }

    # Create object that will be used for GVI function
    mask = {
        'meta': meta,
        'options': options,
        'dsm': dsm,
        'dtm': dtm,
        'green': green
    }

    ### Step 2: Retrieve network, use OSM if not provided
    if network_file is not None:
        # Make sure network file is provided either as geopackage or shapefile
        if os.path.splitext(network_file)[1] not in [".gpkg", ".shp"]:
            raise TypeError("Please provide the network file in '.gpkg' or '.shp' format")
        elif network_file is not None and (os.path.splitext(network_file)[1] == ".gpkg"):
            graph_projected_edges = gpd.read_file(network_file, layer='edges')
        else: 
            graph_projected_edges = gpd.read_file(network_file)

        # Make sure network file has same CRS as poi file
        if not graph_projected_edges.crs.to_epsg() == epsg:
            print("Adjusting CRS of Network file to match with Point of Interest CRS...")
            graph_projected_edges.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")

        # Check if house locations are within network file provided
        bbox_network = graph_projected_edges.unary_union.envelope
        if not all(geom.within(bbox_network) for geom in poi['geometry']):
            raise ValueError("Not all points of interest are within the network file provided, please make sure they are and re-run the function")
    else:
        # Determine polygon that contains total bounds of poi file, incl. buffer distance if specified
        if buffer_dist is None:
            poi_polygon = sg.box(*poi.total_bounds)
        else:
            poi_polygon = sg.box(*poi.total_bounds).buffer(buffer_dist)
        # Transform to 4326 for OSM
        polygon_gdf_wgs = gpd.GeoDataFrame(geometry=[poi_polygon], crs=f"EPSG:{epsg}").to_crs("EPSG:4326") 
        # Extract polygon in EPSG 4326
        wgs_polygon = polygon_gdf_wgs['geometry'].values[0] 

        print(f"Retrieving network within total bounds of {geom_type}(s) of interest, extended by the buffer_dist in case provided...")
        start_network_retrieval = time()
        # Extract network from OpenStreetMap
        network_graph = ox.graph_from_polygon(wgs_polygon, network_type='all', retain_all=True)
        # Project network to original poi file CRS
        graph_projected = ox.project_graph(network_graph, to_crs=f"EPSG:{epsg}")
        # Save network edges in dataframe
        graph_projected_edges = ox.graph_to_gdfs(graph_projected, nodes=False, edges=True)
        end_network_retrieval = time()
        elapsed_network_retrieval = end_network_retrieval - start_network_retrieval
        print(f"Done, running time: {str(timedelta(seconds=elapsed_network_retrieval))} \n")

    ### Step 3: Define sample points on road network for calculating GVI scores
    print("Computing sample points for roads within area of interest's network...")
    start_sample_points = time()
    # Get road sample locations based on sample_dist
    poi['sampled_points'] = poi.apply(lambda row: get_network_sample_points(df_row=row, network_edges=graph_projected_edges, buffer_dist=buffer_dist, sample_dist=sample_dist), axis=1)
    # Explode the 'sampled_points' column so that each point location is stored in new row
    sampled_points_exploded = poi.explode('sampled_points')[['id','sampled_points']].reset_index(drop=True).rename(columns={'sampled_points': 'geometry'})
    sampled_points_gdf = gpd.GeoDataFrame(sampled_points_exploded, crs=f'EPSG:{epsg}').reset_index(drop=True)
    end_sample_points = time()
    elapsed_sample_points = end_sample_points - start_sample_points
    print("Note: creation of sample points based on code by Ondrej Mlynarcik \nsource: https://github.com/Spatial-Data-Science-and-GEO-AI-Lab/2.5D-GreenViewIndex-Netherlands/blob/main/sample_points_linestrings.ipynb")
    print(f"Done, running time: {str(timedelta(seconds=elapsed_sample_points))} \n")

    ### Step 4: Perform the Viewshed GVI calculation
    poi[['GVI', 'nr_of_points', 'GVI_list']] = poi.apply(lambda row: pd.Series(calc_gvi(mask, row, geom_type)), axis=1)
    # Assign GVI scores to each sample road location
    sampled_points_gdf['GVI'] = poi['GVI_list'].explode().reset_index(drop=True)
    print("Note: calculation of Viewshed GVI based on code by Johnny Huck and Labib Labib \nsource: https://github.com/jonnyhuck/green-visibility-index/blob/master/gvi.py \n")

    # Drop irrelevant columns
    poi.drop(['sampled_points', 'GVI_list'], axis=1, inplace=True)

    if write_to_file:
        print("Writing results to new geopackage file in specified directory...")
        # Create output directory if the one specified by user does not yet exist
        os.makedirs(output_dir, exist_ok=True)
        # Ask for filename to use
        input_filename = input("Enter filename for results: ")
        poi.to_file(os.path.join(output_dir, f"{input_filename}.gpkg"), driver="GPKG")
        sampled_points_gdf.to_file(os.path.join(output_dir, f"{input_filename}_sampled_points.gpkg"), driver="GPKG")
        print("Done")

    return poi, sampled_points_gdf

In [7]:
poi, sampled_points_df = get_viewshed_GVI(point_of_interest=multi_point_file,
                                          greendata_raster_file=filepath+"GM_GS_5m.tif",
                                          dtm_raster_file=filepath+"GM_DTM_5m.tif",
                                          dsm_raster_file=filepath+"GM_DSM_5m.tif",
                                          buffer_dist=100,
                                          viewing_dist=250,
                                          sample_dist=50,
                                          observer_height=1.7,
                                          output_dir=results_path)

Retrieving network within total bounds of Point(s) of interest, extended by the buffer_dist in case provided...
Done, running time: 0:00:13.899827 

Computing sample points for roads within area of interest's network...
Note: creation of sample points based on code by Ondrej Mlynarcik 
source: https://github.com/Spatial-Data-Science-and-GEO-AI-Lab/2.5D-GreenViewIndex-Netherlands/blob/main/sample_points_linestrings.ipynb
Done, running time: 0:00:00.126952 



Calculating GVI for Point 3: 100%|██████████| 46/46 [00:05<00:00,  8.70it/s]


Note: calculation of Viewshed GVI based on code by Johnny Huck and Labib Labib 
source: https://github.com/jonnyhuck/green-visibility-index/blob/master/gvi.py 

Writing results to new geopackage file in specified directory...
Done


In [8]:
poi

,id,geometry,GVI,nr_of_points
0,1,POINT (388644.249 392861.634),0.532,12
1,2,POINT (385981.911 393805.494),0.149,72
2,3,POINT (388631.230 395322.181),0.362,46


In [9]:
sampled_points_df

,id,geometry,GVI
0,1,POINT (388564.100 392907.269),0.582617
1,1,POINT (388579.827 392895.998),0.52047
2,1,POINT (388661.088 392846.149),0.509614
3,1,POINT (388564.100 392907.269),0.582617
4,1,POINT (388661.088 392846.149),0.509614
...,...,...,...
125,3,POINT (388729.860 395335.932),0.338168
126,3,POINT (388743.218 395252.725),0.311971
127,3,POINT (388644.604 395385.631),0.414666
128,3,POINT (388626.529 395386.291),0.435242
